In [1]:
# Import standard libraries
import os
import re
import pandas as pd
import numpy as np
import math
from IPython.display import display

import win32con, win32api, os

# IMPORT OWN FUNCTIONS
from save_data import save_csv, save_xlsx

# FILES:
# Root-folder path
main_path = 'Z:\\Projects\\РФФИ-Норвегия\\Данные\\Экспериментальные данные в лаборатории\\Droplet freezing facility data\\Generating droplets'
loc_path = 'C:\\Users\\M.Vulf\\Desktop\\_Generating droplets'

# Dataframe-name
dataframe_name = 'GENERATING_DATA.csv'
xlsx_name = 'GENERATING_DATA.xlsx'

# Name of folder containing droplet properties
imgproc_folder = 'IMAGE_PROCESSING_DATA'
imgproc_path = os.path.join(loc_path, imgproc_folder)

main_df = pd.read_csv(os.path.join(main_path,'GENERATING_DATA.csv'))

# Read image processing df
im_proc_df = pd.read_csv(os.path.join(loc_path,'IMAGE_PROCESSING_DATA','image_processing.csv'))

print('Done!')

Done!


## Add columns from image_processing.csv

#### Modify image_processing data before merging

In [2]:
# Create calibration df
calibrat_df = main_df[main_df['test_type']!='calibrat'][['test','pixel_size','record_rate']]
# calibrat_df.info()

# im_proc_df.info()
# Merge improc and calibr dataframes
im_proc_df = im_proc_df.merge(calibrat_df, how='left',on='test')
# Getting new columns
im_proc_df['droplet_count'] = round(im_proc_df['autoDropCount'])
im_proc_df['flow_type'] = im_proc_df['autoFlowType'].str.lower()
im_proc_df['droplet_size'] = round(im_proc_df['pixDiameter']*im_proc_df['pixel_size'],2)
im_proc_df['sigma_droplet_size'] = round(im_proc_df['sigmaPixDiameter']*im_proc_df['pixel_size'],2)
im_proc_df['starting_velocity'] = round(im_proc_df['startingPixVelocity']*im_proc_df['pixel_size']*1e-6*im_proc_df['record_rate'],5)
im_proc_df['ending_velocity'] = round(im_proc_df['endingPixVelocity']*im_proc_df['pixel_size']*1e-6*im_proc_df['record_rate'],5)
display(im_proc_df.info())
# take necessary part of image processing df
part_im_proc_df = im_proc_df[['test', 'crop', 'droplet_count', 'flow_type', 'droplet_size', 'sigma_droplet_size', 'starting_velocity', 'ending_velocity']]
display(part_im_proc_df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432 entries, 0 to 431
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   test                 432 non-null    int64  
 1   crop                 432 non-null    int64  
 2   autoFlowType         432 non-null    object 
 3   autoDropCount        432 non-null    int64  
 4   pixDiameter          432 non-null    float64
 5   sigmaPixDiameter     432 non-null    float64
 6   startingPixVelocity  432 non-null    float64
 7   endingPixVelocity    432 non-null    float64
 8   pixel_size           432 non-null    float64
 9   record_rate          432 non-null    float64
 10  droplet_count        432 non-null    int64  
 11  flow_type            432 non-null    object 
 12  droplet_size         432 non-null    float64
 13  sigma_droplet_size   432 non-null    float64
 14  starting_velocity    432 non-null    float64
 15  ending_velocity      432 non-null    flo

None

,test,crop,droplet_count,flow_type,droplet_size,sigma_droplet_size,starting_velocity,ending_velocity
0,280,1,0,no,0.00,0.00,0.00000,0.00000
1,281,1,1,single,1900.52,18.46,0.13128,0.64620
2,282,1,1,single,1599.74,15.02,0.13435,0.65110
3,283,1,1,single,1884.04,18.06,0.12715,0.64558
4,284,2,1,single,1574.78,17.59,0.12335,0.64867


#### Merge image processing results with part of generating data

In [3]:
# get tests that are in generating droplet (create copy)
test_list = list(part_im_proc_df['test'])
# Create a copy of main_df in necessary tests
df = main_df[main_df['test'].isin(test_list)].copy(deep=True)

# Delete columns with None-elements
df = df.dropna(axis=1, how='all')

# Merge improc-data with part of Main_df
merge_df = df.merge(part_im_proc_df, how='outer',on=['test','crop'])
# Filling of crops-NaN parameters (pressure, voltage, etc)
part_df = pd.DataFrame(columns=merge_df.columns)
for testNum in main_df['test'].unique():
    part_df = part_df.append(merge_df[merge_df['test']==testNum].fillna(axis=0,method='ffill'),ignore_index=True)
# delete previous rows
main_df = main_df[~main_df['test'].isin(test_list)]

part_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   test                672 non-null    object 
 1   crop                672 non-null    float64
 2   test_type           672 non-null    object 
 3   voltage             672 non-null    float64
 4   pressure            672 non-null    float64
 5   impulse_time        672 non-null    float64
 6   needle              672 non-null    object 
 7   nozzle              672 non-null    object 
 8   calibrat_test       672 non-null    float64
 9   record_rate         672 non-null    float64
 10  comments            672 non-null    object 
 11  experiment_date     672 non-null    object 
 12  tester              672 non-null    object 
 13  video               672 non-null    object 
 14  pixel_size          672 non-null    float64
 15  impulse_count       672 non-null    float64
 16  droplet_

## Append all imgproc-files in one df

In [4]:
# get files with droplet properties
all_files = os.listdir(imgproc_path)
str_files = ','.join(all_files)
pixprop_files = re.findall(r'\d\d\d\d_pixprop_\d.csv',str_files)

# set test-range for finding
test_range = [1,np.inf]

improc_df = pd.DataFrame()
# in setting test-range: collect data
for file_name in pixprop_files:
    # get number of test
    test_num = int(file_name.split('_')[0])
    if (test_num>=test_range[0]) and (test_num<=test_range[1]):
        print('analyzing',file_name)
        
        # get number of crop
        crop_num = int(file_name.split('_')[-1].split('.')[0])

        # read csv-file
        curr_df = pd.read_csv(os.path.join(imgproc_path,file_name))

        # add test,crop num columns
        curr_df['test'] = test_num
        curr_df['crop'] = crop_num
        # change columns order
        cols = curr_df.columns.to_list()
        new_cols = cols[-2:] + cols[:-2]
        curr_df = curr_df[new_cols]

        # add dataframes
        improc_df = improc_df.append(curr_df, ignore_index=True)
display(improc_df)

analyzing 0281_pixprop_1.csv
analyzing 0282_pixprop_1.csv
analyzing 0283_pixprop_1.csv
analyzing 0284_pixprop_1.csv
analyzing 0284_pixprop_2.csv
analyzing 0285_pixprop_1.csv
analyzing 0285_pixprop_2.csv
analyzing 0286_pixprop_1.csv
analyzing 0286_pixprop_2.csv
analyzing 0287_pixprop_1.csv
analyzing 0287_pixprop_2.csv
analyzing 0288_pixprop_2.csv
analyzing 0288_pixprop_3.csv
analyzing 0289_pixprop_1.csv
analyzing 0290_pixprop_1.csv
analyzing 0290_pixprop_2.csv
analyzing 0291_pixprop_1.csv
analyzing 0291_pixprop_2.csv
analyzing 0292_pixprop_1.csv
analyzing 0292_pixprop_2.csv
analyzing 0293_pixprop_1.csv
analyzing 0293_pixprop_2.csv
analyzing 0294_pixprop_1.csv
analyzing 0294_pixprop_2.csv
analyzing 0295_pixprop_2.csv
analyzing 0295_pixprop_3.csv
analyzing 0297_pixprop_1.csv
analyzing 0298_pixprop_1.csv
analyzing 0299_pixprop_1.csv
analyzing 0299_pixprop_2.csv
analyzing 0300_pixprop_1.csv
analyzing 0300_pixprop_2.csv
analyzing 0301_pixprop_1.csv
analyzing 0301_pixprop_2.csv
analyzing 0302

,test,crop,frameNum,y,x,diameter,area
0,281,1,93,227.117595,371.463361,93.301333,6837
1,281,1,94,234.144359,371.871414,92.064953,6657
2,281,1,95,240.651967,371.838353,91.028803,6508
3,281,1,96,247.778820,371.820627,91.014815,6506
4,281,1,97,256.609522,372.258138,91.489217,6574
...,...,...,...,...,...,...,...
19162,519,2,75,1119.726147,380.927823,81.766624,5251
19163,519,2,76,1149.702667,381.609041,82.046435,5287
19164,519,2,77,1179.875333,382.139893,81.789978,5254
19165,519,2,78,1210.053185,382.747656,81.579551,5227


## Modify and save current imgproc file

In [5]:
# get pixel_sizes & record_rates
calibr_df = main_df[['test','crop','pixel_size','record_rate']]
calibr_df = calibr_df.append(part_df[['test','crop','pixel_size','record_rate']])
improc_df = improc_df.merge(calibr_df,on=['test','crop'],how='left')
improc_df.columns = ['test', 'crop', 'frameNum', 'y_pix', 'x_pix','diameter_pix','area_pix','pixel_size','record_rate']

# time in seconds
improc_df['time'] = improc_df['frameNum']/improc_df['record_rate']
# coordinates in mm
improc_df['y'] = improc_df['y_pix']*improc_df['pixel_size']*1e-3
improc_df['x'] = improc_df['x_pix']*improc_df['pixel_size']*1e-3

# get time with 0 at start point
def change_time(row):
    row['time'] = row['time'] - improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['time'].min()
    row['del_y'] = row['y'] - improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['y'].min()
    row['del_x'] = row['x'] - improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['x'].iloc[0]
    return row

new_improc_df = improc_df.apply(change_time,axis=1)
display(new_improc_df)

,test,crop,frameNum,y_pix,x_pix,diameter_pix,area_pix,pixel_size,record_rate,time,y,x,del_y,del_x
0,281,1,93,227.117595,371.463361,93.301333,6837,20.75,1000.0,0.000,4.712690,7.707865,0.000000,0.000000
1,281,1,94,234.144359,371.871414,92.064953,6657,20.75,1000.0,0.001,4.858495,7.716332,0.145805,0.008467
2,281,1,95,240.651967,371.838353,91.028803,6508,20.75,1000.0,0.002,4.993528,7.715646,0.280838,0.007781
3,281,1,96,247.778820,371.820627,91.014815,6506,20.75,1000.0,0.003,5.141411,7.715278,0.428720,0.007413
4,281,1,97,256.609522,372.258138,91.489217,6574,20.75,1000.0,0.004,5.324648,7.724356,0.611957,0.016492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19162,519,2,75,1119.726147,380.927823,81.766624,5251,21.08,1000.0,0.048,23.603827,8.029959,18.709857,0.514476
19163,519,2,76,1149.702667,381.609041,82.046435,5287,21.08,1000.0,0.049,24.235732,8.044319,19.341762,0.528837
19164,519,2,77,1179.875333,382.139893,81.789978,5254,21.08,1000.0,0.050,24.871772,8.055509,19.977801,0.540027
19165,519,2,78,1210.053185,382.747656,81.579551,5227,21.08,1000.0,0.051,25.507921,8.068321,20.613950,0.552839


In [6]:
# save results
save_name = 'imgproc_detailed.csv'
new_improc_df.to_csv(os.path.join(imgproc_path,save_name),index=False)

## Get analyzed columns from imgproc_detailed.csv and get median values

#### Get characteristics from improc_df (detailed)

In [7]:
improc_df = new_improc_df

# function of getting characteristics from improc_df
part_df['fall_time'] = None
part_df['vertical_path'] = None
part_df['max_horizontal_deviation'] = None

def get_improc_params(row):
    # analyze only Single-droplets rows
    if row['droplet_count'] != 1:
        return row
    # time in seconds
    row['fall_time'] = improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['time'].max()
    # path and deviation in mm
    row['vertical_path'] = improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['del_y'].max()
    row['max_horizontal_deviation'] = improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['del_x'].apply(abs).max()   
    return row

part_df = part_df.apply(get_improc_params, axis=1)
display(part_df[part_df['droplet_count']==1][['droplet_size','sigma_droplet_size',
'starting_velocity','ending_velocity','fall_time','vertical_path', 'max_horizontal_deviation']].describe())

,droplet_size,sigma_droplet_size,starting_velocity,ending_velocity,fall_time,vertical_path,max_horizontal_deviation
count,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000
mean,1692.912722,22.079174,0.147897,0.650373,0.053324,21.052718,0.351672
std,206.867288,6.792530,0.022717,0.013611,0.012803,0.403510,0.254012
min,648.800000,10.430000,0.097320,0.419770,0.038000,18.052404,0.013149
25%,1629.310000,19.165000,0.131110,0.647615,0.052000,20.840954,0.166282
50%,1686.240000,21.110000,0.144170,0.650990,0.053000,21.051969,0.283283
75%,1766.030000,23.615000,0.158890,0.654090,0.054000,21.253725,0.509879
max,2164.150000,115.420000,0.292970,0.664030,0.281000,23.410752,1.331544


#### Get median values for "crop 0" of single droplets
- If only one crop: No droplets.
- If count of crops >=2 and they are all - single: get mean value of last two crops. Type: Single
- If any crop with multi/jet case: get mean value of drop count.Type: Multi

In [8]:
# change crop type to int
part_df['crop'] = part_df['crop'].apply(int) 

# create pivot table of crop count
crop_df = pd.pivot_table(part_df, values=['crop','droplet_count'], index='test', aggfunc='max')

# fill crop_count according to crop_df
def crop_drop_writer(row):
    row['crop_count'] = crop_df[crop_df.index == row['test']]['crop'].values[0]
    # !!!!!!!! DELETE this column below
    row['max_droplet_count'] = crop_df[crop_df.index == row['test']]['droplet_count'].values[0]
    return row
# get count of crops
part_df = part_df.apply(crop_drop_writer, axis=1)

# change crop type to int
part_df['crop'] = part_df['crop'].apply(int) 

# create pivot table of crop count
crop_df = pd.pivot_table(part_df, values=['crop','droplet_count'], index='test', aggfunc='max')

# fill crop_count according to crop_df
def crop_drop_writer(row):
    row['crop_count'] = crop_df[crop_df.index == row['test']]['crop'].values[0]
    # !!!!!!!! DELETE this column below
    row['max_droplet_count'] = crop_df[crop_df.index == row['test']]['droplet_count'].values[0]
    return row
# get count of crops
part_df = part_df.apply(crop_drop_writer, axis=1)

# Function of getting mean values in "single" case
def get_mean_crop_params(row):
    if row['crop'] == 0:
        # classify according to crop_count and max_droplet_count
        if row['crop_count'] < 2:
            row['droplet_count'] = 0
            row['flow_type'] = 'no'
        elif row['crop_count'] >= 2 and row['max_droplet_count'] == 1:
            row['droplet_count'] = 1
            row['flow_type'] = 'single'
        else:
            row['droplet_count'] = row['max_droplet_count']
            if row['droplet_count'] <= 4:
                row['flow_type'] = 'multi'
            elif row['droplet_count'] > 4:
                row['flow_type'] = 'jet'

        # get mean characteristics for single case
        if row['flow_type'] == 'single':
            column_list = ['droplet_size','sigma_droplet_size','starting_velocity','ending_velocity', 'fall_time','vertical_path','max_horizontal_deviation']
            with_crop_list = column_list + ['crop']
            # get last two crop values of this test
            values = part_df[(part_df['test']==row['test'])&(part_df['crop'] != 0)][with_crop_list].sort_values(by=['crop'], ascending=False).head(2)
            # get mean value
            row[column_list] = values[column_list].mean()
    return row

# Get mean values in "single" case
part_df = part_df.apply(get_mean_crop_params, axis=1)
part_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   test                      672 non-null    int64  
 1   crop                      672 non-null    int64  
 2   test_type                 672 non-null    object 
 3   voltage                   672 non-null    float64
 4   pressure                  672 non-null    float64
 5   impulse_time              672 non-null    float64
 6   needle                    672 non-null    object 
 7   nozzle                    672 non-null    object 
 8   calibrat_test             672 non-null    float64
 9   record_rate               672 non-null    float64
 10  comments                  672 non-null    object 
 11  experiment_date           672 non-null    object 
 12  tester                    672 non-null    object 
 13  video                     672 non-null    object 
 14  pixel_size

## Final merging

In [9]:
# add new filled out rows and sort data
main_df = main_df.append(part_df, ignore_index=True)
main_df.sort_values(by=['test','crop'],inplace=True)

# delete last column
main_df = main_df.drop(axis=1,columns='max_droplet_count')

# only date
main_df['experiment_date'] = main_df['experiment_date'].dt.date

print(main_df.info())
display(main_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1387 entries, 0 to 1385
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   test                      1387 non-null   int64  
 1   crop                      1387 non-null   float64
 2   test_type                 1387 non-null   object 
 3   voltage                   1377 non-null   float64
 4   pressure                  1377 non-null   float64
 5   impulse_time              1377 non-null   float64
 6   needle                    1377 non-null   object 
 7   nozzle                    1377 non-null   object 
 8   distance                  0 non-null      float64
 9   calibrat_test             1377 non-null   float64
 10  record_rate               1379 non-null   float64
 11  comments                  801 non-null    object 
 12  experiment_date           1387 non-null   object 
 13  tester                    1387 non-null   object 
 14  video   

,test,crop,test_type,voltage,pressure,impulse_time,needle,nozzle,distance,calibrat_test,...,flow_type,droplet_size,sigma_droplet_size,starting_velocity,ending_velocity,fall_time,vertical_path,max_horizontal_deviation,impulse_count,crop_count
0,28,0.0,calibrat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29,0.0,nozzle rec,115.0,500.0,5000.0,N02-1,NZ10,NaN,28.0,...,single,1078.71,12.11,0.11012,0.70474,0.062,25.808833,0.296174,NaN,NaN
2,29,1.0,nozzle rec,115.0,500.0,5000.0,N02-1,NZ10,NaN,28.0,...,single,1077.44,19.23,0.11012,0.71214,0.062,25.808833,0.251238,NaN,NaN
3,29,2.0,nozzle rec,115.0,500.0,5000.0,N02-1,NZ10,NaN,28.0,...,single,1078.71,16.33,0.12760,0.70719,0.060,25.296699,0.296174,NaN,NaN
4,29,3.0,nozzle rec,115.0,500.0,5000.0,N02-1,NZ10,NaN,28.0,...,single,1081.13,10.12,0.07925,0.70379,0.065,25.812991,0.265897,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,519,1.0,nozzle rec,124.8,3900.0,4000.0,N01-1,NZ11,NaN,340.0,...,single,1721.16,21.16,0.15011,0.65592,0.053,21.423417,0.537856,3.0,2.0
1382,519,2.0,nozzle rec,124.8,3900.0,4000.0,N01-1,NZ11,NaN,340.0,...,single,1735.78,19.36,0.15916,0.65701,0.052,21.265831,0.566360,3.0,2.0
1384,520,0.0,nozzle rec,126.6,3900.0,4000.0,N01-1,NZ11,NaN,340.0,...,multi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0
1386,520,1.0,nozzle rec,126.6,3900.0,4000.0,N01-1,NZ11,NaN,340.0,...,multi,0.00,0.00,0.00000,0.00000,NaN,NaN,NaN,3.0,2.0


In [21]:
# leave only date
main_df['experiment_date'] = pd.to_datetime(main_df['experiment_date'])
main_df['experiment_date'] = main_df['experiment_date'].dt.date

In [22]:
# SAVE IN CSV 
save_name = dataframe_name
save_csv(main_df, main_path, save_name)
# SAVE IN XLSX 
save_name = xlsx_name
save_xlsx(main_df, main_path, xlsx_name)

Are you sure you want to write this DataFrame to main DataBase? Print 'yes'
2021-11-19#1509 GENERATING_DATA.csv
Are you sure you want to write this DataFrame to Excel-file? Print 'yes'


'Done'